In [83]:
import gensim.downloader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
from gensim.models import KeyedVectors
from keras.layers import Dense, Flatten, LSTM, GRU, Bidirectional, Embedding
from keras.models import Sequential
from keras.preprocessing import sequence, text
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, classification_report
from sklearn.model_selection import train_test_split
from spacy.tokenizer import Tokenizer

In [84]:
data = pd.read_csv('data\preprocessed.csv').dropna()
data = data[((data[" 'European'"]==1) | (data["'European'"]==1))]

In [85]:
nlp = spacy.load("en_core_web_sm")
#glove = gensim.downloader.load('glove-twitter-200')
#glove.save("glove.model")
glove = KeyedVectors.load("glove.model")

In [86]:
def get_vector(sent):
    for word in sent.lower().split(' '):
        if word.isalpha() == True:
            try:
                return glove[word].tolist()
            except KeyError:
                return glove['unk'].tolist()
        else:
            return glove['unk'].tolist()

In [87]:
def pad_trunc(data, maxlen):
    new_data = []
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)
    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
        
    return new_data

In [88]:
data['vectors'] =  data['Reviews_cleaned'].apply(lambda x: [get_vector(x)])

In [89]:
X = data['vectors']
y = data["Category"]
X = np.array(pad_trunc(X, 3))

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [91]:
X_train.shape, X_test.shape, X.shape

((10786, 3, 200), (2697, 3, 200), (13483, 3, 200))

In [92]:
model = Sequential()
model.add(Bidirectional(LSTM(32, return_sequences=True, input_shape=(3,200), dropout=0.2, recurrent_dropout=0.2)))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [93]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=16,
                    validation_split=0.2)

Epoch 1/10
540/540 [==============================] - 21s 19ms/step - loss: 0.4010 - accuracy: 0.8601 - val_loss: 0.3914 - val_accuracy: 0.8559
Epoch 2/10
540/540 [==============================] - 9s 17ms/step - loss: 0.3817 - accuracy: 0.8643 - val_loss: 0.3936 - val_accuracy: 0.8563
Epoch 3/10
540/540 [==============================] - 9s 16ms/step - loss: 0.3810 - accuracy: 0.8644 - val_loss: 0.3955 - val_accuracy: 0.8563
Epoch 4/10
540/540 [==============================] - 8s 15ms/step - loss: 0.3816 - accuracy: 0.8644 - val_loss: 0.3911 - val_accuracy: 0.8563
Epoch 5/10
540/540 [==============================] - 9s 16ms/step - loss: 0.3805 - accuracy: 0.8644 - val_loss: 0.3935 - val_accuracy: 0.8563
Epoch 6/10
540/540 [==============================] - 8s 15ms/step - loss: 0.3809 - accuracy: 0.8644 - val_loss: 0.3913 - val_accuracy: 0.8563
Epoch 7/10
540/540 [==============================] - 9s 17ms/step - loss: 0.3807 - accuracy: 0.8644 - val_loss: 0.3912 - val_accuracy: 0.856

In [94]:
y_pred = np.round(model.predict(X_test))
cohen_kappa_score(y_test, y_pred)

0.5591194621996916

In [95]:
f1_score(y_test, y_pred)

0.919855920756416

In [96]:
model = Sequential()
model.add(Bidirectional(GRU(32, return_sequences=True, input_shape=(3,200), dropout=0.2, recurrent_dropout=0.2)))
model.add(GRU(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=16,
                    validation_split=0.2)

Epoch 1/10
540/540 [==============================] - 21s 18ms/step - loss: 0.3950 - accuracy: 0.8607 - val_loss: 0.3931 - val_accuracy: 0.8563
Epoch 2/10
540/540 [==============================] - 9s 16ms/step - loss: 0.3823 - accuracy: 0.8644 - val_loss: 0.3916 - val_accuracy: 0.8563
Epoch 3/10
540/540 [==============================] - 9s 16ms/step - loss: 0.3816 - accuracy: 0.8644 - val_loss: 0.3944 - val_accuracy: 0.8563
Epoch 4/10
540/540 [==============================] - 8s 15ms/step - loss: 0.3815 - accuracy: 0.8644 - val_loss: 0.3935 - val_accuracy: 0.8559
Epoch 5/10
540/540 [==============================] - 8s 16ms/step - loss: 0.3807 - accuracy: 0.8644 - val_loss: 0.3921 - val_accuracy: 0.8563
Epoch 6/10
540/540 [==============================] - 9s 16ms/step - loss: 0.3809 - accuracy: 0.8644 - val_loss: 0.3915 - val_accuracy: 0.8563
Epoch 7/10
540/540 [==============================] - 8s 15ms/step - loss: 0.3805 - accuracy: 0.8644 - val_loss: 0.3975 - val_accuracy: 0.855

In [97]:
y_pred = np.round(model.predict(X_test))
cohen_kappa_score(y_test, y_pred)

0.5591194621996916

In [98]:
f1_score(y_test, y_pred)

0.919855920756416